# **TEAM HORUS TASK 2 REPORT**

# Introduction

## Optimization-based White-box attack

Our group opted to do task 2 option 1, i.e. generate adversarial examples for the [Vanilla Athena](https://arxiv.org/pdf/2001.00308.pdf), using optimization-based white-box attack. In the White-box threat model, adversary is aware of the target classifier, existense of defense, weak defenses and the ensemble strategy of the target model.

In this task, we aim to generate adversarial examples using FGSM and PGD in the context of White-box threat model, based on the Expectation Over Transformation (EOT) apporach and evaluate the effectiveness of the crafted adversarial examples on the defense ensemble built in [Vanilla Athena](https://arxiv.org/pdf/2001.00308.pdf), the undefended model and the state-of-the-art PGD Adversarial Trained (PGD-ADT) models.

# Background
In this task, we implemented two different adversarial attacks, namely: 
   * Fast Gradient Sign Method (FGSM) 
   * Projected Gradient Descent (PGD).

## 1. Fast Gradient Sign Method ([FGSM](https://arxiv.org/abs/1412.6572)) Attack 

Fast Gradient Sign Method ([FGSM](https://arxiv.org/abs/1412.6572)) Attack is one of the most popular adversarial attacks. While the target model tries to minimize the loss function by adjusting the weights based on the backpropagated gradients, this powerful attack adds pertubation to original input by maximizing the loss function based on the same backpropagated gradients. That is, it produces adversarial examples by adding the original input image ($x$) and the gradients of the neural network of the loss function ($J$) with respect to input data ($\nabla_xJ(\theta,x,y)$).

It is an attack for a $l_\infty$-bounded adversary that computes an adversarial example as using the following expression:
           
$$ x' = x + \epsilon .sign(\nabla_xJ(\theta,x,y))$$ 
where:
* $x'$ is the adversarial image
* $x$ is the original input image 
* $y$ is the original input label 
* $\theta$  is the model parameter 
* $J$ is the loss function
* $\epsilon$ is the magnitude of the perturbation
  
In essense, this attack can be described as a simple one-iteration scheme for maximizing the inner part of the saddle point formulation.

## 2. Projected Gradient Descent ([PGD](https://arxiv.org/pdf/1706.06083.pdf)) Attack

The Projected Gradient Descent ([PGD](https://arxiv.org/pdf/1706.06083.pdf)) Attack is a more powerful adversary than the FGSM. As its name suggests, it applies a projected gradient descent on negative loss function. In otherwords, it performs an iteration to find the perturbation that maximises the loss of a model on a particular input while keeping the size of the perturbation smaller than a specific amount specified as epsilon, $\epsilon$.
This method processes adversarial examples as follows until a certain stopping criterion is satisfied:
 
$$ x^{t+1} = x^{t} - \Pi_{x+S} (x^t + \alpha sign(\nabla_xJ(\theta,x,y)))$$
where:
* $t$  is the number of iteration
and other parameters are as defined above.

It is similar to the BIM attack (also know as the Iterative [FGSM](https://arxiv.org/abs/1412.6572)), however, what differentiates them is that the PGD initializes the example to a random point within the available set, as decided by the $l_\infty$ norm, and also the PGD does random restart.

The [PGD](https://arxiv.org/pdf/1706.06083.pdf) Attack is considered the most complete adversary as it is not limited by any constraints on time  and effort the adversary expends on optimization  to obtain the best attack by iteration.



## Expectation Over Transformation (EOT) approach:
 
EOT algorithm, is a general frame work allowing for the construction of adversarial examples that remain adversarial over a chosen transformation distribution.

When making the adversarial attacks (AEs) in the white-box case, we have a set of possible classes $Y$ and a space of valid inputs $X$ to the classifier. For any class of $y \in Y$ and input $x \in X$, the function of $P(y|x)$ and its gradient $\nabla_xP(y|x)$ are accessible. The AEs can be produced by maximizing the log-likelihood of the target class $y_t$ over a $\epsilon$-radius ball around the original image (which is showing as a vector of $d$-pixel each in [0,1]): 

$$ argmax \; logP(y_t|x')$$

$$ x'$$

$$ subject \; to \; ||x'-x||_p<\epsilon$$

$$ x' \in \; [0,1]^d$$

Although this approach were effective in generating the adversarial examples, under image transformations these examples fail to be adversarial. That is why the Expecting Over Transformation (EOT) is defined to fix this issue. The EOT is modeling the peturbations whith optimizing procedure. 

EOT does not optimize the log-likelihood of a single example, instead it uses a chosen distribution $T$ of transformation functions $t$. It takes an input $x'$ that is controlled by the adversary to the true input $t(x')$. EOT limits the effective distance between the adversarial and original inputs:

$$\delta = E_{t\sim T} [d(t(x'),t(x))]$$


# Experiment and Settings
 
## Objective
To generate adversarial examples in the context of the **White-box** threat model and evaluate the performance of a built defense ensemble in the [Vanilla Athena](https://arxiv.org/pdf/2001.00308.pdf) against the crafted AEs and further compare it to the performance on the Undefended Model and the state-of-the-art *PGD Adversarial Trained* (PGD-ADT).


## Subsampling
The input files for this project are images of handwritten numbers in the set `{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}` and their true labels contained in the **MNIST** dataset provided. The full **MNIST** dataset contains **10000** samples which could be used to generate the adversarial examples. However, due to limitations in computational time and resources, a smaller data of samples and their labels were generated by sampling the full **MNIST** dataset at a ratio of 1:10 (`0.1`) to ensure consistency and allow for proper comparison and these subsamples and their corresponding labels were used to generate the adversarial examples.

For consistency and proper comparison, the subsamples and corresponding labels generated for ``Task 1`` were used for ``Task 2``

The generated subsample files (containing 1000 samples) for benign sample and their respected labels are located at:
* samples - ``data/subsamples-mnist-ratio_0.1-289385.328.npy`` 
* labels  - ``data/sublabels-mnist-ratio_0.1-289385.328.npy``

These subsample files were used to generate the adversarial examples (AEs) for **White-box** FGSM and PGD adversarial attacks based on the EOT Approach.


## Crafting of Adversarial Examples

### The Targetet Model

Unlike the **Zero-knowledge** AEs in which the attacks are launched at the **Undefended model**, the targeted model for the **White-box attack** is an Ensemble of [**Vanilla Athena**](https://arxiv.org/pdf/2001.00308.pdf) with a **pool of ``20``selected Weak Defenses** whose configuration numbers are: [``1``, ``4``, ``8``, ``10``, ``12``, ``17``, ``20``, ``25``, ``30``, ``35``, ``39``, ``43``, ``48``, ``50``, ``55``, ``57``, ``60``, ``65``, ``68``, ``72``] with the **AVEP** (Average Output based-on Probability) Ensemble strategy.


The AEs were generated as follows:

### [FGSM](https://arxiv.org/abs/1412.6572)
For **[FGSM](https://arxiv.org/abs/1412.6572) attack**, 12 AEs were generated in total: 

For each of two (``2``)variants of **EOT** settings, 6 AEs were generated by varying the values of epsilon ($\epsilon$) as thus: ``0.08``, ``0.15``, ``0.20``, ``0.25``, ``0.30`` and ``0.40``.

In this task, the settings for the **EOT** are specified in ``attack-zk-mnist.json``with a key-value pair: key --- ``"distribution"``; value --- dictionary configuration for the distribution.
  
The two variants of **EOT** used for [FGSM](https://arxiv.org/abs/1412.6572) are:
   *  Rotation with 100 number of samples - a sample attack configuration is:
  
          "configs0": {
            "attack": "fgsm",
            "description": "FGSM_eps0.08_rot_Task2",
            "eps": 0.08,
            "distribution": {
              "num_samples": 100,
              "transformation": "rotation",
              "min_angle": -45,
              "max_angle": 45
          }
          
          
   *  Translation with 100 number of samples - a sample attack configuration is:
  
          "configs6": {
            "attack": "fgsm",
            "description": "FGSM_eps0.08_tran_Task2",
            "eps": 0.08,
            "distribution": {
              "num_samples": 100,
              "transformation": "translation",
              "min_offset": -0.2,
              "max_offset": 0.2
          }
       


### [PGD](https://arxiv.org/pdf/1706.06083.pdf)
Similar to the **[FGSM](https://arxiv.org/abs/1412.6572)**, for **[PGD](https://arxiv.org/pdf/1706.06083.pdf) attack**, 12 AEs were generated in total: 

For each of two (``2``)variants of **EOT** settings, 6 AEs were generated by varying the values of epsilon ($\epsilon$) as thus: ``0.08``, ``0.15``, ``0.20``, ``0.25``, ``0.30`` and ``0.40``.

  
The two variants of **EOT** used for [FGSM](https://arxiv.org/abs/1412.6572) are:
   *  Translation with 50 number of samples - a sample attack configuration is:
  
          "configs12": {
            "attack": "pgd",
            "description": "PGD_eps0.08_tran_50_Task2",
            "eps": 0.08,
            "distribution": {
              "num_samples": 50,
              "transformation": "translation",
              "min_offset": -0.2,
              "max_offset": 0.2
          }
          
          
   *  Translation with 100 number of samples - a sample attack configuration is:
  
          "configs18": {
            "attack": "pgd",
            "description": "PGD_eps0.08_tran_100_Task2",
            "eps": 0.08,
            "distribution": {
              "num_samples": 100,
              "transformation": "translation",
              "min_offset": -0.2,
              "max_offset": 0.2
          }

The settings for these attacks are defined in the ``config/attack-zk-mnist.json`` file.

* NOTE: _The choice of small number of samples was due to time and computational resources constraints_


## Evaluations of Crafted AEs and Ensemble Strategies.
The generated AEs were evaluated on the undefended model, the model with [Vanilla Athena](https://arxiv.org/pdf/2001.00308.pdf) defense and the model with state-of-the-art defense PGD-ADT. 
The [Vanilla Athena](https://arxiv.org/pdf/2001.00308.pdf) defense was tested by changing the number of weak defenses (20 and 40) and the ensemble strategy (Average output based Probability and Majority Voting).

The configuration number for the selected weak defences are:
* [Vanilla Athena](https://arxiv.org/pdf/2001.00308.pdf) with 20 WDs: [``1``, ``4``, ``8``, ``10``, ``12``, ``17``, ``20``, ``25``, ``30``, ``35``, ``39``, ``43``, ``48``, ``50``, ``55``, ``57``, ``60``, ``65``, ``68``, ``72``]
* [Vanilla Athena](https://arxiv.org/pdf/2001.00308.pdf) with 40 WDs: [``1``, ``3``, ``4``, ``5``, ``6``, ``8``, ``10``, ``12``, ``14``, ``15``, ``16``, ``17``, ``20``, ``22``, ``25``, ``26``, ``28``, ``30``, ``32``, ``35``, ``36``, ``39``, ``40``, ``43``, ``45``, ``47``, ``48``, ``50``, ``51``, ``53``, ``55``, ``57``, ``59``, ``60``, ``62``, ``65``, ``67``, ``68``, ``70``, ``72``]

The settings for WDs are defined in the ``config/athena-mnist.json`` file. 

Error rates for all the three types of models were calculated to determine the success rate of the White-box adversarial examples. 


# Relevant Files
* Subsamples and sublabels generated 1:10 ratio can be found at:
 * `samples - /Task2_update/Submission/data/subsamples-mnist-ratio_0.1-289385.328.npy`
 * `labels - Task2_update/Submission/data/sublabels-mnist-ratio_0.1-289385.328.npy`.


* Attack configuration file for the generation of AEs can be found at:
 * `/Task2_update/Submission/config/attack-zk-mnist.json`.
 
 
* Ensemble Configuration for [Vanilla Athena](https://arxiv.org/pdf/2001.00308.pdf) can be found at:
 * `/Task2_update/Submission/config/athena-mnist.json`.
 
 
* Jupyter notebook containing the log of experiment:
 * `/Task2_update/Submission/Team_Horus_Task_2_Generate_WB_AEs.ipynb`.
 
 
* Jupyter notebook used for evaluation of AEs and plotting of graphs:
 * `/Task2_update/Submission/Evaluate_AEs_Task2.ipynb`.
 
 
* Jupyter notebook used for evaluation of AEs and plotting of graphs:
 * `/Task2_update/Submission/results/task_2_results.xlsx`.

# Results and Discussion

# FGSM Attack in the Context of White-box Threat Model



## FGSM with Rotation EOT

### Plot of Sample FGSM (with Rotation setting in EOT) AEs
$\epsilon$: 0.08  |  $\epsilon$: 0.15 | $\epsilon$: 0.20 | $\epsilon$: 0.25 | $\epsilon$: 0.30  | $\epsilon$: 0.40
:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|
![](images/FGSM_eps0.08_rot_Task2.png)  |  ![](images/FGSM_eps0.15_rot_Task2.png) |  ![](images/FGSM_eps0.20_rot_Task2.png) |  ![](images/FGSM_eps0.25_rot_Task2.png) |  ![](images/FGSM_eps0.30_rot_Task2.png) |  ![](images/FGSM_eps0.40_rot_Task2.png)

### Evaluation Results with 20 and 40 WDs Using AVEP and MV Ensemble Strategy

20 WDs AVEP Ensemble Strategy  |  40 WDs AVEP Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/FGSM_with_Rot_20_AVEP.png)  |  ![](images/FGSM_with_Rot_40_AVEP.png) |

20 WDs MV Ensemble Strategy  |  40 WDs MV Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/FGSM_with_Rot_20_MV.png)  |  ![](images/FGSM_with_Rot_40_MV.png) |


### Discussion

The AEs generated by FGSM with Rotation settings in the EOT appear more distorted with the increasing value of epsilon. 
For all variations of ensemble strategy and number of weak defenses,  [Athena](https://arxiv.org/pdf/2001.00308.pdf) seems to have a higher error rate compared to PGD-ADT except for epsilon values around ``0.08`` or less.

Here changing the number of weak defenses from ``20`` to ``40`` or/and ensemble strategy from **MV** to **AVEP** doesn't seem to have any any effect to for the defense against the attack by athena. Since the adversary knows about the athena defense, the PGD-ADT defense is better than athena, except for the case where value of epsilon is ``0.08``.

## FGSM with Translation EOT

### Plot of Sample FGSM (with Rotation setting in EOT) AEs
$\epsilon$: 0.08  |  $\epsilon$: 0.15 | $\epsilon$: 0.20 | $\epsilon$: 0.25 | $\epsilon$: 0.30  | $\epsilon$: 0.40
:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|
![](images/FGSM_eps0.08_tran_Task2.png)  |  ![](images/FGSM_eps0.15_tran_Task2.png) |  ![](images/FGSM_eps0.20_tran_Task2.png) |  ![](images/FGSM_eps0.25_tran_Task2.png) |  ![](images/FGSM_eps0.30_tran_Task2.png) |  ![](images/FGSM_eps0.40_tran_Task2.png)

### Evaluation Results with 20 and 40 WDs Using AVEP and MV Ensemble Strategy

20 WDs AVEP Ensemble Strategy  |  40 WDs AVEP Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/FGSM_with_Tran_20_AVEP.png)  |  ![](images/FGSM_with_Tran_40_AVEP.png) |

20 WDs MV Ensemble Strategy  |  40 WDs MV Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/FGSM_with_Tran_20_MV.png)  |  ![](images/FGSM_with_Tran_40_MV.png) |


### Discussion

The AEs generated by FGSM with Translations settings in the EOT also appear more distorted with the increasing value of epsilon. 
For all variations of ensemble strategy and number of weak defenses,  [Athena](https://arxiv.org/pdf/2001.00308.pdf) seems to have a higher error rate compared to PGD-ADT except for epsilon values of ``0.08`` or less.

Here, changing the ensemble strategy from **MV** to **AVEP** doesn't seem to have any any effect on the **[Athena](https://arxiv.org/pdf/2001.00308.pdf) ensemble's** defense against the attack. However, increasing number of **weak defenses** from ``20`` to ``40`` yields an appreciable reduction in error rates of AEs for high epsilon ($\epsilon$) values. 

Since the adversary knows about the athena defense, the PGD-ADT defense is better than athena, except for the case where value of epsilon is equalt to or less than ``0.08``.





# PGD Attack in the Context of White-box Threat Model






## PGD with Translation and 50 Number of Samples (EOT)

### Plot of Sample PGD (with Translation and 50 Number of Samples setting in EOT) AEs
$\epsilon$: 0.08  |  $\epsilon$: 0.15 | $\epsilon$: 0.20 | $\epsilon$: 0.25 | $\epsilon$: 0.30  | $\epsilon$: 0.40
:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|
![](images/PGD_eps0.08_tran_50_Task2.png)  |  ![](images/PGD_eps0.15_tran_50_Task2.png) |  ![](images/PGD_eps0.20_tran_50_Task2.png) |  ![](images/PGD_eps0.25_tran_50_Task2.png) |  ![](images/PGD_eps0.30_tran_50_Task2.png) |  ![](images/PGD_eps0.40_tran_50_Task2.png)

### Evaluation Results with 20 and 40 WDs Using AVEP and MV Ensemble Strategy

20 WDs AVEP Ensemble Strategy  |  40 WDs AVEP Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/PGD_Tran_50s_20_AVEP.png)  |  ![](images/PGD_Tran_50s_40_AVEP.png) |

20 WDs MV Ensemble Strategy  |  40 WDs MV Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/PGD_Tran_50s_20_MV.png)  |  ![](images/PGD_Tran_50s_40_MV.png) |



### Discussion

The AEs generated by PGD with Translations and 50 number of samples in the **EOT** settings also appear more distorted with the increasing value of epsilon. 
For all variations of ensemble strategy and number of weak defenses,  [Athena](https://arxiv.org/pdf/2001.00308.pdf) seems to have a higher error rate compared to PGD-ADT for epsilon values ($\epsilon$) of ``0.20`` and more.

Changing the ensemble strategy from **MV** to **AVEP** doesn't seem to have any any effect on the **[Athena](https://arxiv.org/pdf/2001.00308.pdf) ensemble's** defense against the attack. However, increasing number of **weak defenses** from ``20`` to ``40`` yields an appreciable reduction in error rates of AEs for high epsilon ($\epsilon$) values. 

Since the adversary knows about the athena defense, the PGD-ADT defense is better than athena, except for the case where value of epsilon is equalt to or less than ``0.08``.


## PGD with Translation and 100 Number of Samples (EOT)

### Plot of Sample PGD (with Translation and 100 Number of Samples setting in EOT) AEs
$\epsilon$: 0.08  |  $\epsilon$: 0.15 | $\epsilon$: 0.20 | $\epsilon$: 0.25 | $\epsilon$: 0.30  | $\epsilon$: 0.40
:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|
![](images/PGD_eps0.08_tran_100_Task2.png)  |  ![](images/PGD_eps0.15_tran_100_Task2.png) |  ![](images/PGD_eps0.20_tran_100_Task2.png) |  ![](images/PGD_eps0.25_tran_100_Task2.png) |  ![](images/PGD_eps0.30_tran_100_Task2.png) |  ![](images/PGD_eps0.40_tran_100_Task2.png)

### Evaluation Results with 20 and 40 WDs Using AVEP and MV Ensemble Strategy

20 WDs AVEP Ensemble Strategy  |  40 WDs AVEP Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/PGD_Tran_100s_20_AVEP.png)  |  ![](images/PGD_Tran_100s_40_AVEP.png) |

20 WDs MV Ensemble Strategy  |  40 WDs MV Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/PGD_Tran_100s_20_MV.png)  |  ![](images/PGD_Tran_100s_40_MV.png) |



### Discussion

Similar to the PGD AES with 50 number of samples, the AEs generated by PGD with Translations and 100 number of samples in the **EOT** settings also appear more distorted with the increasing value of epsilon.

For all variations of ensemble strategy and number of weak defenses,  [Athena](https://arxiv.org/pdf/2001.00308.pdf) seems to have a higher error rate compared to PGD-ADT for epsilon values ($\epsilon$) of ``0.20`` and more.

Changing the ensemble strategy from **MV** to **AVEP** doesn't seem to have any any effect on the **[Athena](https://arxiv.org/pdf/2001.00308.pdf) ensemble's** defense against the attack. However, increasing number of **weak defenses** from ``20`` to ``40`` yields an appreciable reduction in error rates of AEs for high epsilon ($\epsilon$) values. 

Since the adversary knows about the athena defense, the PGD-ADT defense is better than athena, except for the case where value of epsilon is equalt to or less than ``0.08``.

It is also noteworthy that the PGD AEs with 100 number of samples were more successful at fooling [Athena](https://arxiv.org/pdf/2001.00308.pdf) and other models as they had higher error rates, compare to the PGD AEs with 50 number of samples. Hence, with higher number of samples, PGD attack can craft **stronger White-box adversarial examples**.




# Comparison of White-Box and Zero-Knowledge Adversarial Examples




#  White-box FGSM AEs vs Zero-Knowledge FGSM AEs

### Evaluation Results with 20 and 40 WDs Using AVEP and MV Ensemble Strategy

20 WDs AVEP Ensemble Strategy  |  40 WDs AVEP Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/FGSM_comp_ZK_20_AVEP.png)  |  ![](images/FGSM_comp_ZK_40_AVEP.png) |

20 WDs MV Ensemble Strategy  |  40 WDs MV Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/FGSM_comp_ZK_20_MV.png)  |  ![](images/FGSM_comp_ZK_40_MV.png) |



## White-box PGD AEs vs Zero-Knowledge PGD AEs

### Evaluation Results with 20 and 40 WDs Using AVEP and MV Ensemble Strategy

20 WDs AVEP Ensemble Strategy  |  40 WDs AVEP Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/PGD_comp_ZK_20_AVEP.png)  |  ![](images/PGD_comp_ZK_40_AVEP.png) |

20 WDs MV Ensemble Strategy  |  40 WDs MV Ensemble Strategy
:-------------------------:|:-------------------------:|
![](images/PGD_comp_ZK_20_MV.png)  |  ![](images/PGD_comp_ZK_40_MV.png) |




### Discussion

Contrary to what we expected, the error rate of both the **FGSM** and **PGD** **White-box attack** were lower compared to those of the **Zero-knowledge attack**. We expected that **White-box** AEs would be more successful at fooling the [Athena](https://arxiv.org/pdf/2001.00308.pdf) defense than the **Zero-knowledge** AEs

We suspect this may be a result of the lesser number of samples we have used to generate the **White-box** AEs due to computational time and resources constraints.



# Comparison of Generated White-box AEs with Baseline AEs provided




# FGSM

## White-box FGSM AEs vs Baseline PGD AEs

**20 WDs AVEP Ensemble Strategy**
![](images/FGSM_comp_Base_20_AVEP.png)  

**40 WDs AVEP Ensemble Strategy**
![](images/FGSM_comp_Base_40_AVEP.png)




# PGD

## White-box PGD AEs vs Baseline PGD AEs

**20 WDs AVEP Ensemble Strategy**
![](images/PGD_comp_Base_20_AVEP.png)  

**40 WDs AVEP Ensemble Strategy**
![](images/PGD_comp_Base_40_AVEP.png)

### Discussion

For all cases with different number of WDs and ensemble strategy similar trends are observed. The error in the baseline AEs are very high compared to the AEs generated by our group. The error of the generated AEs increase with the values of epsilon as expected.

# Conclusions

1. The defense by [Athena](https://arxiv.org/pdf/2001.00308.pdf) is weaker compared to the PGD-ADT as expected except for cases where epsilon is really low (e.g. 0.08), since the adversary knows about the [Athena](https://arxiv.org/pdf/2001.00308.pdf)'s defense.


2. With higher number of samples, stronger adversarial examples are crafted in the context of the **White-box** Threat model.


3. However, unexpectedly the error rates in the **Zero-knowledge** AEs are higher compared to the **White-box** AEs. It should be the otherway around.


4. Errors in baseline AEs are very high compared to the generated AEs.



## Suggestion for Future Work

* For this work, we selected Weak Defenses (WDs) at random, for future work, a learning startegy could be developed to choose optimum weak defenses that suits a particular set of AEs

# Member's Contribution


All members contributed to the success of task2.

``Rasika (Rasika-prog)``
 * Tuning of codes to generate Adversarial Examples
 * Evaluation of generated Adversarial Examples
 * Code editting of the evaluation function to take another argument which is the dictionary key for Adversarial Examples generated allowing it to be called outside the function to evaluate different categories of AEs (White-box AEs, Zero Knowledge AEs from Task 1 and Baseline AEs provided).
 * Compilation and Analysis of results and report
 * Management of Group meetings
 
``Olajide (42n8dzydoo)``
 * Tuning of codes to generate Adversarial Examples
 * Evaluation of generated Adversarial Examples
 * Code editting of the evaluation function to take another argument which is the dictionary key for Adversarial Examples generated allowing it to be called outside the function to evaluate different categories of AEs (White-box AEs, Zero Knowledge AEs from Task 1 and Baseline AEs provided).
 * Compilation and Analysis of results and report
 * Plotting of the graphs
 * Formatting of report in Markdown
 
``Joshua (ojihjo)``
 * Tuning of codes to generate Adversarial Examples
 * Code debugging
 * Compilation of introduction and literatures for report
 * Compilation and Analysis of results and report
 * Co-management of Group meetings
 
``Kaveh (kavehshariati)``
 * Tuning of codes to generate Adversarial Examples
 * Code debugging
 * Compilation of introduction and literatures for report
 * Compilation and Analysis of results and report
 * Co-management of Group meetings

# Citation

* Ian J. Goodfellow, Jonathon Schlens, and Christian Szegedy. Explaining and harnessing adversarial examples.  In _International Conferences on Learning Representations (ICLR)_. [_arXiv:1412.6572_](https://arxiv.org/pdf/1412.6572.pdf), 2015.


* Madry,A., Makelov, A., Schmidt, L., Tsipras, D., and Vladu, A. Towards Deep Learning Models Resistant to Adversarial Attacks. [_arXiv:1706.06083_](https://arxiv.org/pdf/1706.06083.pdf), 2019.


* Alex Kurakin, Ian J. Goodfellow, Samy Bengio. Adversarial Machine Learning at Scale. In _International Conference on Learning Representation  (ICLR)_. [_arXiv:1611.01236v2_](https://arxiv.org/pdf/1611.01236.pdf?source=post_page---------------------------), 2017.


* Anish Athalye, Logan Engstrom,  Andrew Ilyas, and Kevin Kwok. Synthesizing Robust Adversarial Examples. [_arXiv:1707.07397_](https://arxiv.org/pdf/1707.07397.pdf), 2018.
    * code GitHub repo: [EOT](https://github.com/prabhant/synthesizing-robust-adversarial-examples)


* Ying Meng, Jianhai Su, Jason M. O’Kane, Pooyan Jamshidi. [ATHENA](https://arxiv.org/pdf/2001.00308.pdf): A Framework based on Diverse Weak Defenses for Building Adversarial Defense. _[arXiv:2001.00308v2](https://arxiv.org/pdf/2001.00308.pdf)_, 2020.
     * code GitHub repo: [ATHENA](https://github.com/csce585-mlsystems/project-athena)
